# Complete guide

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata



import trecs
from trecs.models import ContentFiltering, ImplicitMF
from trecs.random import Generator
from trecs.metrics import MSEMeasurement, RecSimilarity, InteractionSimilarity, RecallMeasurement
from trecs.components import Users
import trecs.matrix_ops as mo #Note, in order for the ideal recommender to result in non-insane results, the normalize_users parameter in mo.inner_product must be set to False

In [ ]:
#https://web.ma.utexas.edu/users/mks/statmistakes/skeweddistributions.html

In [ ]:
class IdealRecommender(ContentFiltering):
    """
    With the Ideal Recommender, we make the *strong assumption* that the true scores are provided
    to the recommender system through a custom scoring function, which always returns the true
    underlying user-item scores. Therefore, this class is pretty much an empty skeleton; the only
    modification is that we don't update any internal state of the recommender at each time step.
    
    Amy: Note to self, as a part of the baseline ContentFiltering class, if an actual_item_representation 
    """
    def __init__(self, *args, **kwargs):
        
        
        super().__init__(*args, **kwargs)
    

    def _update_internal_state(self, interactions):
        # do not change users_hat!
        pass


In [ ]:
#np.random.seed(1)
plt.style.use('seaborn')
plt.rcParams.update({'font.size': 50})

In [202]:
# Keep the dimensions small for easy visualization
number_of_users = 15
number_of_attributes = 10
number_of_items = 20
# We define user_representation using the standard integer generator in Numpy.
# We assume a number of interactions with each attribute in the interval [0,4).

users_shape = (number_of_users, number_of_attributes)
actual_user_representation = Users(size=users_shape, num_users=number_of_users)
model_user_representation = np.random.randint(4, size=(number_of_users, number_of_attributes))

# We define item_representation using the Generator that comes with the framework
# We assume a binary matrix with a binomial distribution

actual_item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)

model_item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)

In [ ]:
#Not positive this is correct

content = ContentFiltering(
    num_users=number_of_users,
    num_items=number_of_items,
    num_attributes=number_of_attributes,
    actual_user_representation=actual_user_representation,
    user_representation=model_user_representation, 
    actual_item_representation = actual_item_representation,
    item_representation=actual_item_representation, #model has the true item values
)
# add an MSE measurement
content.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
content.run(timesteps=1)

In [ ]:
ideal = IdealRecommender(
    num_users=number_of_users,
    num_items=number_of_items,
    num_attributes=number_of_attributes,
    actual_user_representation=actual_user_representation,
    user_representation=actual_user_representation.actual_user_profiles.value, 
    actual_item_representation = actual_item_representation,
    item_representation=actual_item_representation, #model has the true item values
    num_items_per_iter="all",
)
# add an MSE measurement
ideal.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
ideal.run(timesteps=5)

In [203]:
mf = ImplicitMF(
    num_users=number_of_users,
    num_items=number_of_items,
    num_latent_factors=number_of_attributes,
    actual_user_representation=actual_user_representation,
    actual_item_representation = actual_item_representation,
    num_items_per_iter=10,
)
# add an MSE measurement
mf.add_metrics(MSEMeasurement(), RecallMeasurement(k=5))
# Run for 5 time steps
#mf.run(timesteps=5,train_between_steps=True)

In [204]:
mf.run(timesteps=1)

100%|██████████| 1/1 [00:00<00:00, 141.40it/s]


In [207]:
#Testing recall stuff. Keep this in until a larger dataset can be evaluated
#k=2
k=5
recommender = mf
shown_item_scores = np.take_along_axis(recommender.predicted_scores.value, recommender.items_shown, axis=1)

shown_item_ranks = np.argsort(shown_item_scores, axis=1)
#top_item_idxs = shown_item_ranks[:, recommender.items_shown.shape[1] - k:]
top_item_idxs = shown_item_ranks[:, -k:]

top_k_items = np.take_along_axis(recommender.items_shown, top_item_idxs, axis=1)

#np.take_along_axis(recommender.users.actual_user_scores.value, top_item_ids, axis=1)

print('items shown:')
print(recommender.items_shown)

print('shown item scores:')
print(shown_item_scores)

print('shown item ranks:')
print(shown_item_ranks)

print('top k item ids')
print(top_item_idxs)

print('top k items:')
print(top_k_items)

interactions = recommender.interactions.reshape(recommender.num_users, 1)
print(interactions)
recall = len(np.where(interactions == top_k_items)[0]) / recommender.num_users
print(recall)

items shown:
[[15  4 17  1  6 11  2  3 18  5]
 [ 1  9 17  2  3  7  5 19  8  6]
 [ 0 10 14  9  6  3  4  8 18 17]
 [13 19  5 14  2 15 18  1  9  8]
 [11  0  8 13  9 15  5 10  4 12]
 [ 0  6 14 16 18 12  2 15  3 19]
 [ 2 12 14  0 15  6 13 16 17 19]
 [16 12  9  1 10 18 19  7  8 13]
 [ 0 11  4 18  3 10  8 14  6 13]
 [15 19 13  2 17  5  4  7  1 14]
 [ 0  4 15 11 17  3 10 14 13  8]
 [16 12  6 14 18 13  2 19  0 15]
 [ 2  1  3 14 19  9  6  5 18 16]
 [11  8 13  4  7 16  0 12 17  5]
 [16 12 13 14  7 19  6 18  2 10]]
shown item scores:
[[ 2.42591308  2.27489656  1.29349126  0.49216367  0.40664089  0.27980714
   0.13765724  0.09902688 -0.12677321 -0.14426996]
 [ 1.63689668  1.24908194  1.20957248  1.0938576   1.04858252  0.81411158
   0.69836725  0.62544259  0.20473443  0.04296785]
 [ 2.97096988  1.94276861  1.03605634  0.98679941  0.67671259  0.34364815
   0.28761303  0.28488361  0.10519972 -0.07951523]
 [ 1.5004487   1.44129148  1.37182779  0.71709371  0.60081034  0.5261027
   0.46040504  0.1976813

In [220]:
print(recommender.items_shown[0:3,:])
print(top_k_items)
print(recommender.predicted_scores.value[0:3,:])

np.savetxt("predicted_scores.csv", recommender.predicted_scores.value, delimiter=",")


[[15  4 17  1  6 11  2  3 18  5]
 [ 1  9 17  2  3  7  5 19  8  6]
 [ 0 10 14  9  6  3  4  8 18 17]]
[[ 6  1 17  4 15]
 [ 3  2 17  9  1]
 [ 6  9 14 10  0]
 [ 2 14  5 19 13]
 [ 9 13  8  0 11]
 [18 16 14  6  0]
 [15  0 14 12  2]
 [10  1  9 12 16]
 [ 3 18  4 11  0]
 [17  2 13 19 15]
 [17 11 15  4  0]
 [18 14  6 12 16]
 [19 14  3  1  2]
 [ 7  4 13  8 11]
 [ 7 14 13 12 16]]
[[-0.2383659   0.49216367  0.13765724  0.09902688  2.27489656 -0.14426996
   0.40664089 -0.25993857 -1.29737349 -0.3208579  -0.4465261   0.27980714
  -0.78423818 -0.47254428 -1.06937814  2.42591308 -0.44005341  1.29349126
  -0.12677321 -0.29754701]
 [-2.86480918  1.63689668  1.0938576   1.04858252 -0.02010522  0.69836725
   0.04296785  0.81411158  0.20473443  1.24908194 -0.09087694 -0.79835804
  -0.43184669 -1.50598339 -0.36585695  0.00669352 -0.54514974  1.20957248
  -0.75869657  0.62544259]
 [ 2.97096988 -1.02729729 -0.62913309  0.34364815  0.28761303 -1.20369821
   0.67671259 -1.7334786   0.28488361  0.98679941  1.9427

In [166]:
user_ids = np.arange(0, recommender.num_users)
item_ids = np.arange(0, recommender.num_items)
user_item_ids= np.dstack(np.meshgrid(item_ids, user_ids)).reshape(-1, 2)
long_actual_scores = np.reshape(recommender.users.actual_user_scores.value, (recommender.num_users*recommender.num_items, 1))
long_predicted_scores = np.reshape(recommender.predicted_scores.value, (recommender.num_users*recommender.num_items, 1))

scores_df = pd.DataFrame(np.hstack((user_item_ids, long_actual_scores, long_predicted_scores)), columns=['item_id', 'user_id', 'actual_scores', 'predicted_scores'])

scores_df.head(16)

In [201]:
shown_item_actual_rel = np.take_along_axis(recommender.users.actual_user_scores.value, recommender.items_shown, axis=1)

#the presence of negative relavance scores creates some issues. Add back the minimum value per user to eliminate negatives
min_value_shown = np.reshape(np.min(shown_item_actual_rel, axis=1), (-1,1))

shown_item_actual_rel += abs(min_value_shown)

shown_item_predicted_rel = np.take_along_axis(recommender.predicted_scores.value, recommender.items_shown, axis=1)

#flip so rank 1 corresponds to the highest value
shown_item_predicted_rank = shown_item_predicted_rel.shape[1] - rankdata(shown_item_predicted_rel, axis=1)
shown_item_actual_rank = shown_item_predicted_rel.shape[1] - rankdata(shown_item_actual_rel, axis=1)

print(shown_item_actual_rel)
print(shown_item_actual_rank)

print(shown_item_predicted_rel)
print(shown_item_predicted_rank)

dcg = np.sum(shown_item_actual_rel / np.log2(shown_item_predicted_rank+2), axis=1)
idcg = np.sum(shown_item_actual_rel / np.log2(shown_item_actual_rank+2), axis=1)

ndcg = dcg / idcg

print(dcg)
print(idcg)
print(ndcg)


[[0.         0.62669818 0.87292633]
 [0.84352619 0.13365747 0.        ]
 [0.12704873 0.         0.32997882]
 [0.5481165  0.         0.77158909]
 [1.09866841 0.         1.2114165 ]]
[[2. 1. 0.]
 [0. 1. 2.]
 [1. 2. 0.]
 [1. 2. 0.]
 [1. 2. 0.]]
[[2.05797979 1.64575314 0.34811002]
 [2.29881233 1.03274101 0.8351257 ]
 [1.19390273 0.7576043  0.69827807]
 [1.40563245 1.22016358 1.05309238]
 [1.08373781 0.91814404 0.84650509]]
[[0. 1. 2.]
 [0. 1. 2.]
 [0. 1. 2.]
 [0. 1. 2.]
 [0. 1. 2.]]
[0.8318657  0.92785467 0.29203814 0.93391104 1.70437666]
[1.26832886 0.92785467 0.41013764 1.1174121  1.90459909]
[0.6558754  1.         0.7120491  0.83578032 0.89487424]


In [ ]:
# Collect measurements about the simulation
results = mf.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
# Collect measurements about the simulation
results = ideal.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
results = content.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
#generate a distribution that hides a subpopulation
number_of_attributes = 10
number_of_maj_users = 150
number_of_min_users = 50

maj_user_representation = np.random.normal(1, 2, size=(number_of_maj_users, number_of_attributes))
min_user_representation = np.random.normal(0.5, 1.25, size=(number_of_min_users, number_of_attributes))
actual_user_representation = np.vstack((maj_user_representation, min_user_representation))
split_indices=number_of_maj_users

In [ ]:
#If plotted without respect to the subgroups, preference means look more or less normally distributed
plt.hist(actual_user_representation.mean(axis=1))



In [ ]:
#when plotting out mean preferences when accounting for group, we can see a clear distinction in preference
plt.hist(maj_user_representation.mean(axis=1), alpha=.7, color='b')
plt.hist(min_user_representation.mean(axis=1), alpha=0.7, color='r')



In [ ]:
filtering = ContentFiltering(actual_user_representation=actual_user_representation, 
                             num_attributes=number_of_attributes,
                             num_items=500)


mse = MSEMeasurement(diagnostics=True)
recall=RecallMeasurement()

filtering.add_metrics(mse, recall)

filtering.startup_and_train(50)
filtering.run(450)

In [ ]:
mse_diagnostics = filtering.metrics[0].get_diagnostics()
mse_beginning = mse_diagnostics.loc[50:, :]
mse_beginning.head()



In [ ]:
def mse_histogram(model, split_indices=None):
    metric = (
                model.predicted_scores.value.mean(axis=1)- model.users.actual_user_scores.value.mean(axis=1))** 2
    
    colors = ["blue", "orange", "red", "yellow", "green"]
    if split_indices is not None and len(split_indices) > 0:
        splits = [0] + split_indices + [metric.size]
        for i in range(len(splits) - 1):
            values = metric[splits[i] : splits[i + 1]]
            plt.hist(values, alpha=0.7, color=colors[i])
    else:
        plt.hist(metric, bins="auto")
        plt.ylabel("observation count (total n={})".format(metric.size))
        plt.xlabel("mean sqaured error")

In [ ]:
mf.startup_and_train(50)


In [ ]:
mse_histogram(mf)

In [ ]:
mf.run(50)
mf.train()
mse_histogram(mf)

In [ ]:
mf.run(50)
mf.train()
mse_histogram(mf)

In [ ]:
mf.run(50)
mf.train()
mse_histogram(mf)

In [ ]:
content.run(50)
mse_histogram(content)

In [ ]:
content.run(50)
mse_histogram(content)

In [ ]:
print(in_k)
print(not_in_k)
print(len(model.interactions))
k

In [ ]:

np.concatenate((np.ones(len(in_k)), np.zeros(len(not_in_k))), axis=None)

In [ ]:
model=mf
k=5

#split_indices = number_of_maj_users

colors = ["blue", "orange", "red", "yellow", "green"]

shown_item_scores = np.take(model.predicted_scores.value, model.items_shown)
shown_item_ranks = np.argsort(shown_item_scores, axis=1)
top_k_items = np.take(model.items_shown, shown_item_ranks[:, k :])
in_k = (np.where(np.isin(model.interactions, top_k_items))[0])
not_in_k = (np.where(~np.isin(model.interactions, top_k_items))[0])
metric = np.concatenate((np.ones(len(in_k)), np.zeros(len(not_in_k))), axis=None)

plt.hist(metric)

##Amy, implement this pie chart for recall at k
# # Pie chart, where the slices will be ordered and plotted counter-clockwise:
# labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
# sizes = [15, 30, 45, 10]
# explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

# fig1, ax1 = plt.subplots()
# ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
#         shadow=True, startangle=90)
# ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# plt.show()

# maj_population_outcomes = metric[:split_indices]
# min_population_outcomes = metric[split_indices:]

# plt.hist(maj_population_outcomes, color=colors[0])
# plt.hist(min_population_outcomes, color=colors[1])



# if split_indices is not None:
#     splits = [0] + split_indices + [metric.size]
#     for i in range(len(splits) - 1):
#         values = metric[splits[i] : splits[i + 1]]
#         plt.hist(values, alpha=0.7, color=colors[i])

# plt.hist(metric, bins="auto")
# plt.ylabel("observation count (total n={})".format(metric.size))
# plt.xlabel("recall at k")


# 
#     if split_indices is not None and len(split_indices) > 0:
#         splits = [0] + split_indices + [metric.size]
#         for i in range(len(splits) - 1):
#             values = metric[splits[i] : splits[i + 1]]
#             plt.hist(values, alpha=0.7, color=colors[i])
#     else:
#         plt.hist(metric, bins="auto")
#         plt.ylabel("observation count (total n={})".format(metric.size))
#         plt.xlabel("mean sqaured error")

In [ ]:
plt.hist(min_population_outcomes)

In [ ]:
len(min_population_outcomes)

In [ ]:
metric_histogram(filtering)

In [ ]:
metric_histogram(filtering, [number_of_maj_users])

In [ ]:
#generate a bimodal distribution
N=500
mu, sigma = 1.845, 1
mu2, sigma2 = 5.845, 1
X1 = np.random.normal(mu, sigma, N)
X2 = np.random.normal(mu2, sigma2, N)
X_bimodal = np.concatenate([X1, X2])

In [ ]:
# print majority / minority outcome stats
def majority_minority_outcomes(metric, split_index):
    split_indices = [split_index]

        
    maj_mean = metric.last_observation[:split_index].mean()
    maj_std = metric.last_observation[:split_index].std()

    min_mean = metric.last_observation[split_index:].mean()
    min_std = metric.last_observation[split_index:].std()

    print("Majority group statistics: ", maj_mean, "(mean), ", maj_std, "(std)")
    print("Minority group statistics: ", min_mean, "(mean), ", min_std, "(std)")
    print()
    
    metric.hist(split_indices)

In what follows, we expand on this minimal example to gain a deeper understanding of what happens under the hood.

In [ ]:
filtering = ContentFiltering(actual_user_representation=actual_user_representation, 
                             num_attributes=number_of_attributes,
                             num_items=500)


mse = MSEMeasurement(diagnostics=True)
filtering.add_metrics(mse)

filtering.startup_and_train(50)
majority_minority_outcomes(mse, number_of_maj_users)


In [ ]:
filtering.run(450)
majority_minority_outcomes(mse, number_of_maj_users)

In [ ]:
bimodal = plt.hist(X_bimodal, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Bimodal Distribution')
plt.show()

In [ ]:
print(np.mean(X_bimodal))
print(np.std(X_bimodal))

In [ ]:
N=1000
mu, sigma = 14.99, 4
X1 = np.random.normal(mu, sigma, N)
X_skew = np.log2(X1)

skew = plt.hist(X_skew, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Skewed Distribution')
plt.show()

In [ ]:
N=1000
mu, sigma = 3.85, 2.2
normal_dist = np.random.normal(mu, sigma, N)

skew = plt.hist(normal_dist, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Normal Distribution')
plt.show()

In [ ]:
#How to characterize power, type 1 vs type 2 errors 